In [ ]:
#--------Loading in Packages------#
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from scipy.io import mmread
from scipy.sparse import csr_matrix
import seaborn as sns
import os
import tacco as tc

In [ ]:
#--------Loading in Matrix scRNA-seq AnnData-------#
Matrix = sc.read_h5ad("/projects/b1217/HHA/Bulb_Recluster_5_22_AnnData/Matrix_Palantir_6_11_25.h5ad")
#create new column in var as names
Matrix.var["names"] = Matrix.var.index.copy()
Matrix

In [ ]:
#--------Color Dictionaries------#
#Colors for Fine Annotation
fine_colors = {"Lower COL17": "#89C75F",
               "Upper COL17": "#3BBCA8",
               "LPC": "#208A42",
               "Medulla": "#0C727C",
               "Early_Cortex": "#9ECAE1",
               "Middle_Cortex": "#4292C6",
               "Late_Cortex": "#08306B",
               "Early_Cuticle": "#E6C2DC",
               "Middle_Cuticle": "#C06CAB",
               "Late_Cuticle": "#89288F",
               "Early_IRS_I": "#D8A767",
               "Early_IRS_II": "#F47D2B",
               "IRS_Henle": "#F37B7D",
               "IRS_Huxley": "#7E1416",
               "IRS_Cuticle": "#D24B27"}
#Colors for Broad Annotation
broad_colors = {"COL17": "#89C75F",
               "LPC": "#208A42",
               "Cortex": "#4292C6",
               "Cuticle": "#89288F",
               "IRS": "#D24B27"}

#--------Plotting UMAPs------#
#Fine Annotation
sc.pl.umap(Matrix, color="MatrixAnnotationFine", palette = fine_colors,
    frameon=False, size = 15, legend_loc="on data")
#Broad Annotation
sc.pl.umap(Matrix, color="MatrixAnnotationBroad", palette = broad_colors,
           legend_loc="on data", frameon = False, size = 15)
sc.pl.umap(Matrix, color="GeneralAnnotation", legend_loc="on data",
           frameon = False, size = 15)

In [ ]:
#--------Loading in Full scRNA Atlas-----#
HHA = sc.read_h5ad("/projects/b1217/Edward/R_Projects/HHA/h5ad/20250516_HHA-SC-Final.h5ad")
#-----Crossmapping var_names to EnsemblID--------#
symbol_to_ensembl = dict(zip(HHA.var["names"], HHA.var["ensembl"]))
def map_to_ensembl(name):
    if name.startswith("ENSG"):
        return name
    return symbol_to_ensembl.get(name, "Unknown")
Matrix.var["ensembl"] = Matrix.var["names"].apply(map_to_ensembl)

#-------Creating EnsemblID column and set as index--------#
Matrix = Matrix[:, Matrix.var["ensembl"] != "Unknown"].copy()
new_index = Matrix.var["ensembl"]
Matrix.var.index = new_index
Matrix.var_names = new_index
Matrix.var

In [ ]:
#-------Loading in Annotated Spatial Object
adata_sub = sc.read_h5ad("/projects/b1217/HHA/Bulb_Spatial/20250516_HHA_Spatial_TACCO_Matrix_Subset.h5ad")

In [ ]:
matrix_colors_spatial = {"Lower COL17": "#89C75F",
               "Upper COL17": "#3BBCA8",
               "LPC": "#208A42",
               "Medulla": "#0C727C",
               "Early_Cortex": "#9ECAE1",
               "Middle_Cortex": "#4292C6",
               "Late_Cortex": "#08306B",
               "Early_Cuticle": "#E6C2DC",
               "Middle_Cuticle": "#C06CAB",
               "Late_Cuticle": "#89288F",
               "Early_IRS_I": "#D8A767",
               "Early_IRS_II": "#F47D2B",
               "IRS_Henle": "#F37B7D",
               "IRS_Huxley": "#7E1416",
               "IRS_Cuticle": "#D24B27",
                "Unknown": "#00000000"}

#--------Plotting Final TACCO Annotations in Space------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(adata_sub, color=["MatrixAnnotationFine"], img_key="hires", ax=ax, palette = matrix_colors_spatial)
plt.tight_layout()
#Saving as png
fig.savefig("/projects/b1217/HHA/Bulb_Seurat_Plots/Spatial/TACCO_Mapping_Spatial.png",
            dpi = 600, pad_inches = 0.5, bbox_inches = "tight")

In [ ]:
#-------Reading in Gene Trends Across Cortex Pseudotime-------#
cortex_gene_exp_trends = pd.read_csv("/projects/b1217/HHA/Bulb_Recluster_5_22_AnnData/Cortex_Gene_Trends_Pseudotime.csv")
cortex_gene_exp_trends.set_index("Barcode")

In [ ]:
#-------Plotting Upper Pseudotimes----------#
Upper_Matrix = Matrix[Matrix.obs["MatrixAnnotationFine"].isin(["Upper COL17",
                                                        "Early_Cortex", "Middle_Cortex", "Late_Cortex",
                                                       "Early_Cuticle", "Middle_Cuticle", "Late_Cuticle"]),:].copy()
Upper_Matrix

In [ ]:
#------Subsetting Spatial Data for Upper Lineage Cells------#
Spatial_Upper = adata_sub[adata_sub.obs["MatrixAnnotationFine"].isin(["Upper COL17",
                                                        "Early_Cortex", "Middle_Cortex", "Late_Cortex",
                                                       "Early_Cuticle", "Middle_Cuticle", "Late_Cuticle"])].copy()
Spatial_Upper

In [ ]:
#-------Plotting Upper Lineage Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_Upper, color=["MatrixAnnotationFine"], img_key="hires", palette = matrix_colors_spatial, ax=ax)

In [ ]:
#-------Running TACCO Cell x Spot Mapping----#
Upper_Matrix.obs.index = Upper_Matrix.obs_names
Upper_Matrix.obs["cell"] = Upper_Matrix.obs_names
Upper_Matrix.obs["cell"] = pd.Categorical(Upper_Matrix.obs.index)
tc.tl.annotate(adata = Spatial_Upper, reference = Upper_Matrix, annotation_key = 'cell', result_key='cell',multi_center=None)

In [ ]:
#---------Getting Top Mapping Cell for Each Spot--------#
Spatial_Upper.obs["max_cell"] = Spatial_Upper.obsm["cell"].idxmax(axis=1).astype("category")
Spatial_Upper.obs["max_ident"] =Spatial_Upper.obs["max_cell"].map(Upper_Matrix.obs["MatrixAnnotationFine"]).astype("category")
Spatial_Upper.obs["palantir_pseudotime_CoCu"] = Spatial_Upper.obs["max_cell"].map(Upper_Matrix.obs["palantir_pseudotime_CoCu"]).astype("float32")

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_Upper, color=["max_ident"], img_key="hires", palette = matrix_colors_spatial, ax=ax)

In [ ]:
#-------Writing to h5ad-------#
Spatial_Upper.write_h5ad("/projects/b1217/HHA/Bulb_Spatial/20250516_HHA_Spatial_TACCO_Upper_Subset.h5ad")

In [ ]:
#------Subsetting for Lower Pseudotimes------#
#scRNA
Lower_Pops = ["Lower COL17", "Early_IRS_I", "Early_IRS_II",  "IRS_Huxley", "IRS_Henle", "IRS_Cuticle"]
Lower_Matrix = Matrix[Matrix.obs["MatrixAnnotationFine"].isin(Lower_Pops)].copy()
print(Lower_Matrix)
#Spatial
Spatial_Lower = adata_sub[adata_sub.obs["MatrixAnnotationFine"].isin(Lower_Pops)].copy()
print(Spatial_Lower)

In [ ]:
#-------Plotting Lower Lineage Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_Lower, color=["MatrixAnnotationFine"], img_key="hires", palette = matrix_colors_spatial, ax=ax)

In [ ]:
#-------Running TACCO Cell x Spot Mapping----#
Lower_Matrix.obs.index = Lower_Matrix.obs_names
Lower_Matrix.obs["cell"] = Lower_Matrix.obs_names
Lower_Matrix.obs["cell"] = pd.Categorical(Lower_Matrix.obs.index)
tc.tl.annotate(adata = Spatial_Lower, reference = Lower_Matrix, annotation_key = 'cell', result_key='cell',multi_center=None)

In [ ]:
#---------Getting Top Mapping Cell for Each Spot--------#
Spatial_Lower.obs["max_cell"] = Spatial_Lower.obsm["cell"].idxmax(axis=1).astype("category")
Spatial_Lower.obs["max_ident"] =Spatial_Lower.obs["max_cell"].map(Lower_Matrix.obs["MatrixAnnotationFine"]).astype("category")
Spatial_Lower.obs["palantir_pseudotime_IRS"] = Spatial_Lower.obs["max_cell"].map(Lower_Matrix.obs["palantir_pseudotime_IRS"]).astype("float32")

In [ ]:
#-------Plotting IRS Top Idents in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_Lower, color=["max_ident"], img_key="hires", palette = matrix_colors_spatial, ax=ax)

In [ ]:
#-------Plotting IRS Pseudotime in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_Lower, color=["palantir_pseudotime_IRS"], img_key="hires", cmap = "magma", ax=ax)

In [ ]:
#-------Writing to h5ad-------#
Spatial_Lower.write_h5ad("/projects/b1217/HHA/Bulb_Spatial/20250516_HHA_Spatial_TACCO_Lower_Subset.h5ad")

In [ ]:
#-----Subsetting for Cortex Lineage------#
Cortex = Matrix[Matrix.obs["MatrixAnnotationFine"].isin(["Upper COL17", "Early_Cortex", "Middle_Cortex", "Late_Cortex"]),:].copy()
print(Cortex) #8430
#Subsetting for Multiome
Cortex_Multiome = Cortex[Cortex.obs["Barcode"].isin(cortex_gene_exp_trends["Barcode"])].copy()
print(Cortex_Multiome) #8430

In [ ]:
#-----------Saving Cortex Metadata to csv--------------#
Cortex_Multiome.obs.to_csv("/projects/b1217/HHA/Bulb_Spatial/Cortex_Multiome_Subset.csv")

In [ ]:
#-------Loading in Gene Trends-------#
cortex_gene_exp_trends_reordered = pd.read_csv("/projects/b1217/HHA/Bulb_Recluster_5_22_AnnData/Cortex_Gene_Trends_Pseudotime_Reordered.csv")
cortex_gene_exp_trends_reordered["barcode"] = cortex_gene_exp_trends_reordered["Barcode"]
cortex_gene_exp_trends_reordered = cortex_gene_exp_trends_reordered.set_index("barcode") 
cortex_gene_exp_trends_reordered

In [ ]:
#------Addding Data to AnnData----#
Cortex_Multiome.obs["Pseudotime"] = cortex_gene_exp_trends_reordered['Pseudotime'].astype("float32")
Cortex_Multiome.obs["KRT36_GEX"] = cortex_gene_exp_trends_reordered['KRT36_GEX'].astype("float32")
Cortex_Multiome.obs["KRT36_DORC"] = cortex_gene_exp_trends_reordered['KRT36_DORC'].astype("float32")
Cortex_Multiome.obs["KRT36_Lag"] = cortex_gene_exp_trends_reordered['KRT36_Lag'].astype("float32")
Cortex_Multiome.obs["TP63_GEX"] = cortex_gene_exp_trends_reordered['TP63_GEX'].astype("float32")
Cortex_Multiome.obs["TP63_DORC"] = cortex_gene_exp_trends_reordered['TP63_DORC'].astype("float32")
Cortex_Multiome.obs["TP63_Lag"] = cortex_gene_exp_trends_reordered['TP63_Lag'].astype("float32")

In [ ]:
#------Subsetting Spatial Data for Cortical Cells------#
Spatial_Cortex = adata_sub[adata_sub.obs["MatrixAnnotationFine"].isin(["Upper COL17", "Early_Cortex", "Middle_Cortex", "Late_Cortex"])].copy()
Spatial_Cortex

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_Cortex, color=["MatrixAnnotationFine"], img_key="hires", palette = matrix_colors_spatial, ax=ax)

In [ ]:
#-------Rerunning TACCO Cell x Spot Mapping: Cortex Only (No Cuticle)----#
Cortex_Multiome.obs.index = Cortex_Multiome.obs_names
Cortex_Multiome.obs["cell"] = Cortex_Multiome.obs_names
Cortex.obs["cell"] = pd.Categorical(Cortex.obs.index)
tc.tl.annotate(adata = Spatial_Cortex, reference = Cortex_Multiome, annotation_key = 'cell', result_key='cell',multi_center=None)

In [ ]:
#---------Getting Top Mapping Cell for Each Spot--------#
Spatial_Cortex.obs["max_cell"] = Spatial_Cortex.obsm["cell"].idxmax(axis=1).astype("category")
Spatial_Cortex.obs["max_ident"] =Spatial_Cortex.obs["max_cell"].map(Matrix.obs["MatrixAnnotationFine"]).astype("category")

In [ ]:
#-------Adding Gene Dynamics to each Spot-------#
#KRT36
Spatial_Cortex.obs["PT"] =Spatial_Cortex.obs["max_cell"].map(Cortex_Multiome.obs["Pseudotime"]).astype("float32")
Spatial_Cortex.obs["KRT36_GEX"] =Spatial_Cortex.obs["max_cell"].map(Cortex_Multiome.obs["KRT36_GEX"]).astype("float32")
Spatial_Cortex.obs["KRT36_DORC"] =Spatial_Cortex.obs["max_cell"].map(Cortex_Multiome.obs["KRT36_DORC"]).astype("float32")
Spatial_Cortex.obs["KRT36_Lag"] =Spatial_Cortex.obs["max_cell"].map(Cortex_Multiome.obs["KRT36_Lag"]).astype("float32")
#TP63
Spatial_Cortex.obs["TP63_GEX"] =Spatial_Cortex.obs["max_cell"].map(Cortex_Multiome.obs["TP63_GEX"]).astype("float32")
Spatial_Cortex.obs["TP63_DORC"] = Spatial_Cortex.obs["max_cell"].map(Cortex_Multiome.obs["TP63_DORC"]).astype("float32")
Spatial_Cortex.obs["TP63_Lag"] =Spatial_Cortex.obs["max_cell"].map(Cortex_Multiome.obs["TP63_Lag"]).astype("float32")

In [ ]:
#-------------Plotting TP63 DORC Accessibility--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_Cortex, color=["TP63_DORC"], img_key="hires", cmap = "magma", ax=ax)

In [ ]:
#-------------Plotting TP63 GEX--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_Cortex, color=["TP63_GEX"], img_key="hires", cmap = "magma", ax=ax)

In [ ]:
#-------------Plotting TP63 Lag--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_Cortex, color=["TP63_Lag"], img_key="hires", cmap = "turbo", ax=ax)

In [ ]:
#-------Reading in Gene Trends Across IRS Pseudotime-------#
irs_gene_exp_trends = pd.read_csv("/projects/b1217/HHA/Bulb_Recluster_5_22_AnnData/IRSCuticle_Gene_Trends_Pseudotime.csv")

#------Loading in Motif Trends------#
irs_motif_trends_reordered = pd.read_csv("/projects/b1217/HHA/Bulb_Recluster_5_22_AnnData/IRSCuticle_Motif_Deviations_Pseudotime_Reordered.csv")
irs_motif_trends_reordered

In [ ]:
#-----Subsetting for IRS Lineage------#
#Subsetting for Multiome
IRS_Multiome = Matrix[Matrix.obs["Platform"] == "Multiome"].copy()
IRS_Multiome = Matrix[Matrix.obs["MatrixAnnotationFine"].isin(["Lower COL17", "Early_IRS_I", "Early_IRS_II", "IRS_Henle", "IRS_Huxley", "IRS_Cuticle"])].copy()
print(IRS_Multiome) #8430

In [ ]:
#------Subsetting Spatial Data for Cortical Cells------#
Spatial_IRS = adata_sub[adata_sub.obs["MatrixAnnotationFine"].isin(["Lower COL17", "Early_IRS_I", "Early_IRS_II", "IRS_Henle", "IRS_Huxley", "IRS_Cuticle"])].copy()
Spatial_IRS

In [ ]:
#-------Plotting IRS Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["MatrixAnnotationFine"], img_key="hires", palette = matrix_colors_spatial, ax=ax)

In [ ]:
#------Rerunning Annotation using Matrix Anndata Object: IRS Fine Mapping------#
tc.tl.annotate(Spatial_IRS, IRS_Multiome, annotation_key='MatrixAnnotationFine', result_key='MatrixAnnotationFine')

In [ ]:
#-----Retrieving Best Annotation Mapping for Each Cluster------#
tc.utils.get_maximum_annotation(Spatial_IRS, 'MatrixAnnotationFine', result_key='MatrixAnnotationFine')

In [ ]:
#--------Plotting Final TACCO Annotations in Space------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["MatrixAnnotationFine"], img_key="hires", ax=ax, palette = matrix_colors_spatial)
plt.tight_layout()

In [ ]:
#-----Subsetting for IRS Cuticle------#
#Subsetting for Multiome
IRS_Multiome = Matrix[Matrix.obs["Barcode"].isin(irs_gene_exp_trends["Barcode"])].copy()
print(IRS_Multiome) #8430
#------Subsetting Spatial Data for IRS Cuticle------#
Spatial_IRS = Spatial_IRS[Spatial_IRS.obs["MatrixAnnotationFine"].isin(["Lower COL17", "Early_IRS_I", "Early_IRS_II","IRS_Cuticle"])].copy()
print(Spatial_IRS)

In [ ]:
#-------Plotting IRS Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["MatrixAnnotationFine"], img_key="hires", palette = matrix_colors_spatial, ax=ax)

In [ ]:
#-------Running TACCO Cell x Spot Mapping----#
IRS_Multiome.obs.index = IRS_Multiome.obs_names
IRS_Multiome.obs["cell"] = IRS_Multiome.obs_names
IRS_Multiome.obs["cell"] = pd.Categorical(IRS_Multiome.obs.index)
tc.tl.annotate(adata = Spatial_IRS, reference = IRS_Multiome, annotation_key = 'cell', result_key='cell',multi_center=None)

In [ ]:
#---------Getting Top Mapping Cell for Each Spot--------#
Spatial_IRS.obs["max_cell"] = Spatial_IRS.obsm["cell"].idxmax(axis=1).astype("category")
Spatial_IRS.obs["max_ident"] =Spatial_IRS.obs["max_cell"].map(Matrix.obs["MatrixAnnotationFine"]).astype("category")

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["max_ident"], img_key="hires", palette = matrix_colors_spatial, ax=ax)

In [ ]:
#-------------Writing Metadata to csv----------#
IRS_Multiome.obs.to_csv("/projects/b1217/HHA/Bulb_Spatial/IRS_Multiome_Subset.csv")

In [ ]:
#------Addding Data to AnnData----#
#Loading in Gene Dynamics Data
irs_gene_exp_trends_reordered = pd.read_csv("/projects/b1217/HHA/Bulb_Recluster_5_22_AnnData/IRSCuticle_Gene_Trends_Pseudotime_Reordered.csv")

#------GATA3------#
IRS_Multiome.obs["Pseudotime"] = irs_gene_exp_trends_reordered['Pseudotime'].values
IRS_Multiome.obs["GATA3_GEX"] = irs_gene_exp_trends_reordered['GATA3_GEX'].values
IRS_Multiome.obs["GATA3_DORC"] = irs_gene_exp_trends_reordered['GATA3_DORC'].values
IRS_Multiome.obs["GATA3_Lag"] = irs_gene_exp_trends_reordered['GATA3_Lag'].values

#-----TP63------#
IRS_Multiome.obs["TP63_GEX"] = irs_gene_exp_trends_reordered['TP63_GEX'].values
IRS_Multiome.obs["TP63_DORC"] = irs_gene_exp_trends_reordered['TP63_DORC'].values
IRS_Multiome.obs["TP63_Lag"] = irs_gene_exp_trends_reordered['TP63_Lag'].values

In [ ]:
#-------Adding La to each Spot-------#
#Pseudotime
Spatial_IRS.obs["PT"] =Spatial_IRS.obs["max_cell"].map(IRS_Multiome.obs["Pseudotime"]).astype("float32")
#GATA3
Spatial_IRS.obs["GATA3_GEX"] =Spatial_IRS.obs["max_cell"].map(IRS_Multiome.obs["GATA3_GEX"]).astype("float32")
Spatial_IRS.obs["GATA3_DORC"] =Spatial_IRS.obs["max_cell"].map(IRS_Multiome.obs["GATA3_DORC"]).astype("float32")
Spatial_IRS.obs["GATA3_Lag"] = Spatial_IRS.obs["max_cell"].map(IRS_Multiome.obs["GATA3_Lag"]).astype("float32")
#TP63
Spatial_IRS.obs["TP63_GEX"] =Spatial_IRS.obs["max_cell"].map(IRS_Multiome.obs["TP63_GEX"]).astype("float32")
Spatial_IRS.obs["TP63_DORC"] =Spatial_IRS.obs["max_cell"].map(IRS_Multiome.obs["TP63_DORC"]).astype("float32")
Spatial_IRS.obs["TP63_Lag"] = Spatial_IRS.obs["max_cell"].map(IRS_Multiome.obs["TP63_Lag"]).astype("float32")

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["PT"], img_key="hires", cmap = "turbo", ax=ax)

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["GATA3_DORC"], img_key="hires", cmap = "magma", ax=ax)

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["GATA3_GEX"], img_key="hires", cmap = "magma", ax=ax)

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["GATA3_Lag"], img_key="hires", cmap = "magma", ax=ax)

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["TP63_DORC"], img_key="hires", cmap = "magma", ax=ax)

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["TP63_GEX"], img_key="hires", cmap = "magma", ax=ax)

In [ ]:
#-------Plotting Cortex Selection in Space--------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(Spatial_IRS, color=["TP63_Lag"], img_key="hires", cmap = "Greens_r", ax=ax)

In [ ]:
#-------Writing objects to h5ad-------#
#Cortex
Spatial_Cortex.write_h5ad("/projects/b1217/HHA/Bulb_Spatial/20250516_HHA_Spatial_TACCO_Cortex_Subset.h5ad")
#IRS Cuticle
Spatial_IRS.write_h5ad("/projects/b1217/HHA/Bulb_Spatial/20250516_HHA_Spatial_TACCO_IRS_Cuticle_Subset.h5ad")